In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/05/07 22:17:45 WARN Utils: Your hostname, arkadia-355.local resolves to a loopback address: 127.0.0.1; using 10.0.1.12 instead (on interface en0)
23/05/07 22:17:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/07 22:17:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/07 22:17:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
from pyspark.sql import types

In [4]:
schema = types.StructType([
    types.StructField('type', types.StringType(), True), 
    types.StructField('key', types.StringType(), True), 
    types.StructField('revision', types.IntegerType(), True), 
    types.StructField('last_modified', types.TimestampType(), True), 
    types.StructField('json', types.StringType(), True)
])

In [5]:
df = (
    spark.read
    .option("sep", "\t")
    .option("header", "false")
    .schema(schema)
    .csv("ol_dump_authors_latest.txt")
)

In [6]:
df = df.repartition(24)

In [7]:
df.write.parquet("ol/authors/latest", mode="overwrite")

23/05/07 22:18:02 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
23/05/07 22:18:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/05/07 22:18:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/05/07 22:18:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/05/07 22:18:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/05/07 22:18:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Sc

In [8]:
df = spark.read.parquet("ol/authors/latest")

In [9]:
df.printSchema()

root
 |-- type: string (nullable = true)
 |-- key: string (nullable = true)
 |-- revision: integer (nullable = true)
 |-- last_modified: timestamp (nullable = true)
 |-- json: string (nullable = true)



In [10]:
df

DataFrame[type: string, key: string, revision: int, last_modified: timestamp, json: string]

In [11]:
df.show()

+------------+--------------------+--------+--------------------+--------------------+
|        type|                 key|revision|       last_modified|                json|
+------------+--------------------+--------+--------------------+--------------------+
|/type/author| /authors/OL5774906A|       1|2008-10-23 08:03:...|{"name": "Guoxian...|
|/type/author| /authors/OL4946143A|       1|2008-09-22 13:35:...|{"name": "Shi Mao...|
|/type/author| /authors/OL1434023A|       3|2011-06-19 09:07:...|{"photos": [67761...|
|/type/author| /authors/OL9990392A|       1|2021-12-26 06:16:...|{"type": {"key": ...|
|/type/author|/authors/OL10974596A|       1|2022-11-21 15:14:...|{"type": {"key": ...|
|/type/author| /authors/OL4242464A|       1|2008-04-30 20:50:...|{"name": "Syed M....|
|/type/author| /authors/OL6794328A|       1|2010-03-27 04:49:...|{"name": "Mark M....|
|/type/author|/authors/OL11034476A|       1|2022-11-23 15:02:...|{"type": {"key": ...|
|/type/author|/authors/OL10627480A|       1

In [12]:
from pyspark.sql.functions import col, explode, from_json, explode_outer

In [30]:
json_schema = types.StructType([types.StructField("name", types.StringType(), True)])

https://stackoverflow.com/questions/51070251/pyspark-explode-json-in-column-to-multiple-columns

In [31]:
df.withColumn("json", from_json("json", json_schema)).select(
    "type",
    "key",
    "json.name",
    "revision",
    "last_modified"
).show()

+------------+--------------------+--------------------+--------+--------------------+
|        type|                 key|                name|revision|       last_modified|
+------------+--------------------+--------------------+--------+--------------------+
|/type/author| /authors/OL5774906A|        Guoxiang Liu|       1|2008-10-23 08:03:...|
|/type/author| /authors/OL4946143A|            Shi Mao.|       1|2008-09-22 13:35:...|
|/type/author| /authors/OL1434023A|Nathan Altshiller...|       3|2011-06-19 09:07:...|
|/type/author| /authors/OL9990392A|      Thisbe Gensler|       1|2021-12-26 06:16:...|
|/type/author|/authors/OL10974596A|Foreword by Morri...|       1|2022-11-21 15:14:...|
|/type/author| /authors/OL4242464A|      Syed M. Sarwar|       1|2008-04-30 20:50:...|
|/type/author| /authors/OL6794328A|   Mark M. Yarbrough|       1|2010-03-27 04:49:...|
|/type/author|/authors/OL11034476A|North Florida Ama...|       1|2022-11-23 15:02:...|
|/type/author|/authors/OL10627480A|   Alons

`type` is redundant so we can leave that out when selecting pertinent columns